In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
# from reporting.performance import load_reyl_data
# load_reyl_data()

Exception: Exception whilst parsing or saving /home/alex/PycharmProjects/datafeed/features/test_data/reyl_legacy/reyl_transactions_legacy.xlsx |  | Can only use .str accessor with string values!

In [1]:
import wsgi
from pandas import read_csv, to_datetime
from trading.research.mft_backtest import get_query_symbols, get_origin_data_dic, get_bar_price_df, get_restructured_data, \
    add_trading_daytime_cols, create_backtest_df, get_complete_backtest_df, get_txs_pos_dfs, get_pnl_gbp, \
    get_total_daily_pnl, get_perf_matrix
from datetime import time

ENVIRONMENT=DEV, DATABASES={'default': {'ENGINE': 'django.db.backends.sqlite3', 'NAME': 'D:\\ShinyCorp\\db_dev.sqlite3'}}


In [2]:
configs = {'opening_signal_threshold': 0.006, 'closing_signal_threshold': -0.006,
               'dealing_cutoff_time_london': time(11, 0, 0),
               'earliest_trading_time_london': {'TP1 Index': time(10, 55, 0), 'ES1 Index': time(10, 55, 0)},
               'minimum_holding_period_days': 10, 'vol_based_thresholds': True, 'open_threshold_factors': [0.00025, 0],
               'close_threshold_factors': [-0.00025, 0], 'equity_trade': False, 'start_dt': '2014-10-01',
               'end_dt': '2020-03-01', 'geo_settings': {'US': 'ES1 Index'}, 'one_position': True,
               'funds': ['JPMUEAA LN Equity', 'HSBAMCA LN Equity']}

symbol_dic = get_query_symbols(configs)
dic = get_origin_data_dic(symbol_dic, **configs)
bar_df = get_bar_price_df(dic, symbol_dic, dic['universe'], **configs)
restructured_data = get_restructured_data(bar_df, dic['daily_p'], **configs)
data_dic_by_asset = restructured_data[0]
daily_price_df = restructured_data[1]


d:\virtualenvs\datafeed\lib\site-packages\django\db\models\fields\__init__.py:1423: RuntimeWarning: DateTimeField Price.time received a naive datetime (2014-10-01 00:00:00) while time zone support is active.
  RuntimeWarning)
d:\virtualenvs\datafeed\lib\site-packages\django\db\models\fields\__init__.py:1423: RuntimeWarning: DateTimeField Price.time received a naive datetime (2020-03-01 00:00:00) while time zone support is active.
  RuntimeWarning)


In [3]:
add_trading_daytime_cols(data_dic_by_asset, symbol_dic, **configs)
backtest_df = create_backtest_df(data_dic_by_asset, **configs)
backtest_df = get_complete_backtest_df(backtest_df, dic['universe'], **configs)
txs_pos_dic = get_txs_pos_dfs(backtest_df, daily_price_df, dic['universe'], **configs)
pnl_gbp = get_pnl_gbp(backtest_df, txs_pos_dic, daily_price_df, dic['universe'], **configs)
total_daily_pnl_gbp = get_total_daily_pnl(pnl_gbp)
total_cum_pnl_gbp = total_daily_pnl_gbp.cumsum()
total_cum_pnl_gbp.plot(grid=True, figsize=(20, 10))
perf_matrix = get_perf_matrix(total_cum_pnl_gbp)

AttributeError: Can only use .str accessor with string values!

'19:00:00'

d:\virtualenvs\datafeed\lib\site-packages\django\db\models\fields\__init__.py:1423: RuntimeWarning: DateTimeField Price.time received a naive datetime (2019-11-20 00:00:00) while time zone support is active.
  RuntimeWarning)
d:\virtualenvs\datafeed\lib\site-packages\django\db\models\fields\__init__.py:1423: RuntimeWarning: DateTimeField Price.time received a naive datetime (2019-11-19 00:00:00) while time zone support is active.
  RuntimeWarning)


d:\virtualenvs\datafeed\lib\site-packages\django\db\models\fields\__init__.py:1423: RuntimeWarning: DateTimeField Price.time received a naive datetime (2019-05-16 00:00:00) while time zone support is active.
  RuntimeWarning)
d:\virtualenvs\datafeed\lib\site-packages\django\db\models\fields\__init__.py:1423: RuntimeWarning: DateTimeField Price.time received a naive datetime (2019-06-30 00:00:00) while time zone support is active.
  RuntimeWarning)
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "d:\virtualenvs\datafeed\lib\site-packages\IPython\core\interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-23f9b0218138>", line 2, in <module>
    '2019-05-16', '2019-06-30', include_holidays=False)
  File "D:\PythonProjects\datafeed\server\trading\fund_classifier.py", line 311, in calculate_fund_classification
    fx, fx_time, index_offset, include_holidays)
  File "D:\PythonProjects\datafeed\server\app\cache.py", line 23, in _f
    value = f(*args, **kwargs)
  File "D:\PythonProjects\datafeed\server\trading\fund_classifier.py", line 234, in get_prices_for_fund_classifier
    p_df_dic = get_sliced_formatted_data(origin_data, start_dt, end_dt, index_offset_days, include_holidays)
  File "D:\PythonProjects\datafeed\server\trading\fund_classifier.py", line 94, in get_sliced_formatted_data
    sliced_fx_p = sliced_fx_p.at_time(val_time_london)
NameError: name 'val_time_london

NameError: name 'val_time_london' is not defined

TypeError: filter() argument after * must be an iterable, not Q

In [46]:

from sqlalchemy import create_engine
database_url = 'sqlite:///D:\\PythonProjects\\datafeed\\opps\\db_dev.sqlite3'

db_engine = create_engine(database_url, echo=False)




In [23]:
from datetime import datetime, timezone

def save_portaracqg_data_into_db(txt_file_name, db_engine, asset_type, price_type, source, resolution,
                          data_file_path=None, aspects=['open', 'high', 'low', 'close']):
    if data_file_path is not None:
        txt_file_path = data_file_path + txt_file_name
    else:
        txt_file_path = txt_file_name
    symbol = txt_file_name[:-4]
    df = read_csv(txt_file_path, sep=',', header=None, names=['as_of', 'time', 'open', 'high', 'low', 'close', 'volume'])
    dt_index = to_datetime(df['as_of'] * 10000 + df['time'], format='%Y%m%d%H%M', utc=True).floor('s')
    df['as_of'] = to_datetime('today').floor('d')
    df['time'] = dt_index
    df['symbol'] = symbol
    df['source'] = source
    df['resolution'] = resolution
    df['asset_type'] = asset_type
    df['price_type'] = price_type

    for aspect in aspects:
        df['value'] = df[aspect]
        df['aspect'] = aspect
        temp_df = df.drop(['open', 'high', 'low', 'close', 'volume'], axis='columns')
    return temp_df
        # temp_df.to_sql('api_price', con=db_engine, index=False, if_exists='append')


def save_dukescopy_data_into_db(txt_file_name, db_engine, asset_type, price_type, source, resolution,
                          data_file_path=None, aspects=['close']):
    if data_file_path is not None:
        txt_file_path = data_file_path + txt_file_name
    else:
        txt_file_path = txt_file_name
    symbol = txt_file_name[:-8]

    dtype = {'open': float, 'high': float, 'low': float, 'close': float, 'volume': float}
    df = read_csv(txt_file_path, header=1, names=['as_of', 'open', 'high', 'low', 'close'], dtype=dtype)
    df['time'] = to_datetime(df['as_of'], format='%d.%m.%Y %H:%M:%S.000', utc=True).floor('s')
    df['as_of'] = to_datetime('today').floor('d')
    df['symbol'] = symbol
    df['source'] = source
    df['resolution'] = resolution
    df['asset_type'] = asset_type
    df['price_type'] = price_type

    for aspect in aspects:
        df['value'] = df[aspect]
        df['aspect'] = aspect
        temp_df = df.drop(['open', 'high', 'low', 'close'], axis='columns')
    return temp_df
        # temp_df.to_sql('api_price', con=db_engine, index=False, if_exists='append')

In [12]:
import os
# path = 'D:\\ShinyCorp\\PortaraCQG 5m bars\\Futures\\5 Minutes\\futures\\ESH18 Index.txt'
fx_path = 'D:\\ShinyCorp\\Dukescopy 5m fx bars\\GBPJPY Curncy ask.csv'
# list_of_fx = os.listdir(fx_path)

In [49]:
# for fx in list_of_fx:
#     save_dukescopy_data_into_db(fx, db_engine, 'fx_spot', fx[:-4][-3:], 'Dukascopy', '5m', fx_path)

0        2014-09-05 00:05:00+00:00
1        2014-09-05 00:10:00+00:00
2        2014-09-05 00:15:00+00:00
3        2014-09-05 00:20:00+00:00
4        2014-09-05 00:25:00+00:00
                    ...           
592122   2020-04-21 23:35:00+00:00
592123   2020-04-21 23:40:00+00:00
592124   2020-04-21 23:45:00+00:00
592125   2020-04-21 23:50:00+00:00
592126   2020-04-21 23:55:00+00:00
Name: time, Length: 592127, dtype: datetime64[ns, UTC]

In [24]:
fx = 'GBPJPY Curncy ask.csv'
df = save_dukescopy_data_into_db(fx, 'a', 'fx_spot', fx[:-4][-3:], 'Dukascopy', '5m', 'D:\\ShinyCorp\\Dukescopy 5m fx bars\\')


TypeError: data type 'datetime64[s, UTC]' not understood

'sdfadsfg//ESH18 Index'

'dfTPX Index'

9

,as_of,time,symbol,source,resolution,asset_type,price_type,value,aspect
0,2017-03-13 09:35:00+00:00,2017-03-13 09:35:00+00:00,ESH18 Index,test,5m,index_future,trade,2361.00,open
1,2017-03-13 20:10:00+00:00,2017-03-13 20:10:00+00:00,ESH18 Index,test,5m,index_future,trade,2364.00,open
2,2017-03-14 20:10:00+00:00,2017-03-14 20:10:00+00:00,ESH18 Index,test,5m,index_future,trade,2355.50,open
3,2017-03-15 20:10:00+00:00,2017-03-15 20:10:00+00:00,ESH18 Index,test,5m,index_future,trade,2372.75,open
4,2017-03-16 20:10:00+00:00,2017-03-16 20:10:00+00:00,ESH18 Index,test,5m,index_future,trade,2371.25,open
...,...,...,...,...,...,...,...,...,...
28996,2018-03-16 13:05:00+00:00,2018-03-16 13:05:00+00:00,ESH18 Index,test,5m,index_future,trade,1022.00,volume
28997,2018-03-16 13:10:00+00:00,2018-03-16 13:10:00+00:00,ESH18 Index,test,5m,index_future,trade,510.00,volume
28998,2018-03-16 13:15:00+00:00,2018-03-16 13:15:00+00:00,ESH18 Index,test,5m,index_future,trade,775.00,volume
28999,2018-03-16 13:20:00+00:00,2018-03-16 13:20:00+00:00,ESH18 Index,test,5m,index_future,trade,853.00,volume
